In [0]:

from pyspark.sql.functions import row_number
from pyspark.sql.window import Window


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Top3RecentOrders").getOrCreate()

# Sample data
data = [
    (1, 101, "2023-01-15"),
    (1, 101, "2023-07-15"),
    (1, 101, "2023-07-16"),
    (2, 102, "2023-02-20"),
    (1, 103, "2023-03-10"),
    (3, 104, "2023-04-05"),
    (2, 105, "2023-05-12"),
    (2, 105, "2023-05-12")
]

# Define the schema
schema = ["CUSTOMERID", "ORDERID", "ORDERDATE"]

# Create a DataFrame
df = spark.createDataFrame(data, schema)
df.show()


+----------+-------+----------+
|CUSTOMERID|ORDERID| ORDERDATE|
+----------+-------+----------+
|         1|    101|2023-01-15|
|         1|    101|2023-07-15|
|         1|    101|2023-07-16|
|         2|    102|2023-02-20|
|         1|    103|2023-03-10|
|         3|    104|2023-04-05|
|         2|    105|2023-05-12|
|         2|    105|2023-05-12|
+----------+-------+----------+



<h3>Retrieve the top 2 most recent orders for each customer</h3>

In [0]:
win=Window.partitionBy("CUSTOMERID").orderBy(df["ORDERDATE"].desc())
df=df.withColumn("rnk",row_number().over(win))
df.show()
df.filter(df.rnk<3).show()

+----------+-------+----------+---+
|CUSTOMERID|ORDERID| ORDERDATE|rnk|
+----------+-------+----------+---+
|         1|    101|2023-07-16|  1|
|         1|    101|2023-07-15|  2|
|         1|    103|2023-03-10|  3|
|         1|    101|2023-01-15|  4|
|         2|    105|2023-05-12|  1|
|         2|    105|2023-05-12|  2|
|         2|    102|2023-02-20|  3|
|         3|    104|2023-04-05|  1|
+----------+-------+----------+---+

+----------+-------+----------+---+
|CUSTOMERID|ORDERID| ORDERDATE|rnk|
+----------+-------+----------+---+
|         1|    101|2023-07-16|  1|
|         1|    101|2023-07-15|  2|
|         2|    105|2023-05-12|  1|
|         2|    105|2023-05-12|  2|
|         3|    104|2023-04-05|  1|
+----------+-------+----------+---+

